In [ ]:
# HWATA 1: Streamlit App for Regularized Building Footprint Extraction

import streamlit as st
import geopandas as gpd
import rasterio
import numpy as np
import torch
from rasterio import features
from tempfile import NamedTemporaryFile
import os
import gdown
import segmentation_models_pytorch as smp
from shapely.geometry import shape
from buildingregulariser import regularize_geodataframe
import folium
from streamlit_folium import st_folium

st.set_page_config(page_title="HWATA 1 – GeoAI Feature Extractor", layout="centered")
st.title("HWATA 1 – Building Footprint Extractor")
st.markdown("Upload a 30cm satellite or aerial image (GeoTIFF), and HWATA 1 will extract regularized building footprints as GeoJSON.")

# Upload model and image
uploaded_img = st.file_uploader("Upload 30 cm Image (GeoTIFF)", type=["tif", "tiff"])

if uploaded_img:
    with NamedTemporaryFile(delete=False, suffix=".tif") as tmp_img:
        tmp_img.write(uploaded_img.read())
        tmp_img_path = tmp_img.name

    st.success("Image uploaded successfully. Running prediction...")

    # Load image
    with rasterio.open(tmp_img_path) as src:
        image = src.read([1, 2, 3])  # Use only RGB
        transform = src.transform
        height, width = src.height, src.width
        raster_crs = src.crs

    image = image.astype(np.float32) / 255.0

    # Load model (download if missing)
    model_path = "pretrained_unet_building_segmentation.pth"
    if not os.path.exists(model_path):
        gdrive_id = "15ZDjkna10HX17nh-P9fxdxEShh_ZNWdc"
        url = f"https://drive.google.com/uc?id={gdrive_id}"
        gdown.download(url, model_path, quiet=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1
    ).to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    patch_size = 256
    full_pred_mask = np.zeros((height, width), dtype=np.uint8)

    with torch.no_grad():
        for i in range(0, height - patch_size + 1, patch_size):
            for j in range(0, width - patch_size + 1, patch_size):
                patch = image[:, i:i+patch_size, j:j+patch_size]
                if patch.shape[1:] != (patch_size, patch_size):
                    continue
                patch_tensor = torch.tensor(patch, dtype=torch.float32).unsqueeze(0).to(device)
                output = torch.sigmoid(model(patch_tensor))
                pred_patch = (output.squeeze().cpu().numpy() > 0.5).astype(np.uint8)
                full_pred_mask[i:i+patch_size, j:j+patch_size] = pred_patch

    st.success("Prediction completed. Converting to GeoJSON...")

    results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for s, v in features.shapes(full_pred_mask.astype(np.int16), transform=transform)
        if v == 1
    )
    gdf_pred = gpd.GeoDataFrame.from_features(list(results), crs=raster_crs)

    # Filter and regularize
    gdf_proj = gdf_pred.to_crs(gdf_pred.estimate_utm_crs())
    gdf_proj['area_m2'] = gdf_proj.geometry.area
    gdf_filtered = gdf_proj[gdf_proj['area_m2'] >= 10].to_crs(raster_crs)
    polygons = regularize_geodataframe(gdf_filtered)

    st.success("Building footprints extracted and regularized.")

    # Optional preview
    if st.checkbox("Preview Building Footprints (GeoJSON overlay on Google imagery)"):
        st.subheader("Interactive Preview")
        bounds = polygons.total_bounds  # [minx, miny, maxx, maxy]
        center_lat = (bounds[1] + bounds[3]) / 2
        center_lon = (bounds[0] + bounds[2]) / 2

        m = folium.Map(location=[center_lat, center_lon], zoom_start=18)
        folium.TileLayer('https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google Satellite', name='Google Satellite').add_to(m)
        folium.GeoJson(polygons).add_to(m)

        st_data = st_folium(m, width=700, height=500)

    geojson_str = polygons.to_json()
    st.download_button("\ud83d\udcc5 Download Building Footprints (GeoJSON)", geojson_str, file_name="building_footprints.geojson", mime="application/geo+json")

    os.remove(tmp_img_path)